In [1]:
!pip install dask-ml

In [2]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
train_data, test_data, train_targets, test_targets = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)

In [4]:
# Create tuples hidden_layer_sizes
hidden_layer_tuples = []
n_layers = [1, 2]
n_neurons = [10, 20, 30, 40, 50]
for layers in n_layers:
    for neurons in n_neurons:
        cur_size = (neurons,) * layers
        hidden_layer_tuples.append(cur_size)
print('The first hidden layer tuples:')
print(hidden_layer_tuples)

The first hidden layer tuples:
[(10,), (20,), (30,), (40,), (50,), (10, 10), (20, 20), (30, 30), (40, 40), (50, 50)]


In [5]:
def calculate_min(scores):
    test_scores = []
    min_test_score = np.inf
    for score in scores:
        if score < min_test_score:
            min_test_score = score
        test_scores.append(min_test_score)
    return test_scores

In [6]:
from dask_ml.model_selection import HyperbandSearchCV
from distributed import Client
client = Client()

all_val_scores = []
all_test_scores = []
n_iter = 100
num_folds=2
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

param_grid_hyperband = {
    'hidden_layer_sizes': hidden_layer_tuples,
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': np.random.uniform(0.00001, 0.99999, 1000),
    'batch_size': [*range(2, 300, 1)],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': np.random.uniform(0.00001, 0.99999, 1000),
}

for random_state in range(5):
    reg = HyperbandSearchCV(MLPRegressor(solver='adam', random_state=42), param_grid_hyperband, scoring='neg_mean_squared_error', max_iter=200, verbose=1, random_state=random_state)
    reg.fit(train_data.to_numpy(), train_targets)
    
    reg_test_score = mean_squared_error(test_targets, reg.predict(test_data))
    all_test_scores.append(reg_test_score)

    print("Best MSE {:.3f} params {}".format(-reg.best_score_, reg.best_params_))

    all_val_scores.append(calculate_min(-reg.cv_results_['test_score']))

[CV, bracket=4] creating 81 models
[CV, bracket=3] creating 34 models
[CV, bracket=2] creating 15 models
[CV, bracket=1] creating 8 models
[CV, bracket=0] creating 5 models
[CV, bracket=0] For training there are between 323 and 323 examples in each chunk
[CV, bracket=1] For training there are between 323 and 323 examples in each chunk
[CV, bracket=0] validation score of -118.2629 received after 1 partial_fit calls
[CV, bracket=1] validation score of -106.4621 received after 1 partial_fit calls
[CV, bracket=2] For training there are between 323 and 323 examples in each chunk
[CV, bracket=2] validation score of -95.7872 received after 1 partial_fit calls
[CV, bracket=3] For training there are between 323 and 323 examples in each chunk
[CV, bracket=0] validation score of -21.3930 received after 200 partial_fit calls
[CV, bracket=2] validation score of -95.4549 received after 22 partial_fit calls
[CV, bracket=4] For training there are between 323 and 323 examples in each chunk
[CV, bracket

In [7]:
df = pd.DataFrame(all_val_scores).transpose()
df.head()

,0,1,2,3,4
0,497.258860,120.545038,6.619066e+06,140.390631,525.821272
1,497.258860,120.545038,1.127681e+02,140.390631,175.093419
2,232.651031,120.545038,1.127681e+02,140.390631,175.093419
3,192.525066,120.545038,1.127681e+02,140.390631,175.093419
4,95.076248,120.545038,1.127681e+02,140.390631,175.093419


In [9]:
mean_std = []
for row_index in range(len(df)):
    row_np = df.iloc[row_index,:].to_numpy()
    mean_row = np.mean(row_np)
    std_row = np.std(row_np)
    mean_std.append((mean_row, std_row))
mean_std

[(1324069.9877942824, 2647497.9731954113),
 (209.21121568714074, 145.62602946970827),
 (156.28964981059477, 43.838843637880586),
 (148.26445677774353, 30.884129566274513),
 (128.7746933074631, 27.349138641163005),
 (96.58447785244746, 26.23678390369322),
 (96.58447785244746, 26.23678390369322),
 (96.58447785244746, 26.23678390369322),
 (93.8718567887059, 21.91216767182873),
 (93.22786129636324, 22.502439826171102),
 (93.22786129636324, 22.502439826171102),
 (90.07216203958653, 20.56373338034815),
 (90.07216203958653, 20.56373338034815),
 (90.07216203958653, 20.56373338034815),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.53789860727908, 19.614657435206848),
 (89.39477122217848, 19.74729830123448

In [10]:
final_df = pd.DataFrame(mean_std, columns=['Mean', 'Std'])
final_df

,Mean,Std
0,1.324070e+06,2.647498e+06
1,2.092112e+02,1.456260e+02
2,1.562896e+02,4.383884e+01
3,1.482645e+02,3.088413e+01
4,1.287747e+02,2.734914e+01
...,...,...
138,2.796881e+01,1.092904e+01
139,2.621045e+01,8.554956e+00
140,2.621045e+01,8.554956e+00
141,2.621045e+01,8.554956e+00


In [11]:
final_df.to_csv('Hyperband - Final Results.csv', index=False)

In [12]:
test_df = pd.DataFrame(all_test_scores, columns=['Test scores'])
test_df.to_csv('Hyperband - Test scores.csv', index=False)